## Data Platform Team Co-op Case Study - Andrew Zang

Submitted by: Andrew Zang

Email: andrewxzang@gmail.com

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

Split the data by "\n" and store the list in columns

Note that the last element in `row` will be empty, so we pop from the rows

In [3]:
rows = data.split("\n")
rows.pop()

''

split each row of record by ";" and store the 2D list in columns_data

In [4]:
rows_data = []
for row in rows:
    rows_data.append((row.split(";")))

first row of columns_data is the column names, and store the other rows in row_data

In [5]:
columns_name = rows_data[0]
rows_data = rows_data[1:]

Create a dataframe named table with preprocessed data

In [6]:
table = pd.DataFrame(rows_data, columns=columns_name)
table

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


Check the types on the dataframe

In [7]:
table.dtypes

Airline Code    object
DelayTimes      object
FlightCodes     object
To_From         object
dtype: object

Convert the FlightCodes into number from str

In [8]:
table['FlightCodes'] = pd.to_numeric(table['FlightCodes'])
table

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


fill all null values in FlightCodes

In [9]:
# backward fill
table['FlightCodes'] = table.FlightCodes.bfill() - \
                       table.groupby(table.FlightCodes.notnull().cumsum()).cumcount() * 10
# forward fill
table['FlightCodes'] = table.FlightCodes.ffill() + \
                       table.groupby(table.FlightCodes.notnull().cumsum()).cumcount() * 10
table

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],20025.0,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045.0,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


convert `FlightCodes` to Integer

In [10]:
table['FlightCodes'] = table['FlightCodes'].astype(int)
table

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,london_MONTreal


Check types of dataframe

In [11]:
table.dtypes

Airline Code    object
DelayTimes      object
FlightCodes      int64
To_From         object
dtype: object

Seperate the To_From column into two seperate columns To and From, make all letters uppercase

In [12]:
table[['To', 'From']] = table["To_From"].str.upper().str.split("_", expand=True)
table

,Airline Code,DelayTimes,FlightCodes,To_From,To,From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork,WATERLOO,NEWYORK
1,<Air France> (12),[],20025,Montreal_TORONTO,MONTREAL,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa,CALGARY,OTTAWA
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER,OTTAWA,VANCOUVER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,london_MONTreal,LONDON,MONTREAL


Clean the airline_codes to have no punctuation except spaces in the middle

In [13]:
alphabet_and_space = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def keep_only_alphabet_and_space(string):
    """
        this function only keeps alphabet and space in a string, used for Airline Code col
    """
    filtered_string = ''.join(filter(alphabet_and_space.__contains__, string))
    return filtered_string

In [14]:
table["Airline Code"] = table['Airline Code'].apply(keep_only_alphabet_and_space)
table

,Airline Code,DelayTimes,FlightCodes,To_From,To,From
0,Air Canada,"[21, 40]",20015,WAterLoo_NEWYork,WATERLOO,NEWYORK
1,Air France,[],20025,Montreal_TORONTO,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALgary_Ottawa,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,Ottawa_VANcouvER,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,london_MONTreal,LONDON,MONTREAL


A new table is generated from the raw data



In [15]:
table = table[['Airline Code', 'DelayTimes', 'FlightCodes', 'To', 'From']]
table

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL
